In [1]:
import mpg.mpgml.dataset.generator as mpgml_generator

2023-04-02 04:57:54.603900: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [1]:
from importlib import reload
import mpg.wrapper as mpgwrapper

In [32]:
import numba
import numpy as np
import tensorflow as tf
import mpg.graph.random_graph
import networkx as nx
import itertools
import tensorflow_probability as tfb
import mpg.wrapper as mpgwrapper


def _convert_sparse_matrix_to_sparse_tensor(X, shape_hint):
    coo = X.tocoo()
    indices = np.mat([coo.row, coo.col]).transpose()
    return tf.SparseTensor(indices, coo.data, shape_hint)


def _stack_sparse_tensors(shape_hint, *A):
    indices = []
    values = []
    for i, Z in enumerate(A):
        for L, v in zip(Z.indices, Z.values):
            indices.append([i, *L])
            values.append(v)
    return tf.sparse.SparseTensor(indices, values, shape_hint)


def _as_tensor(A, as_dense: bool, shape_hint=None):
    if as_dense:
        return tf.convert_to_tensor(A.todense())
    else:
        return _convert_sparse_matrix_to_sparse_tensor(A, shape_hint)


def _generate_instances(n, p, seed, cardinality: int, target: bool, as_graph: bool,
                        adj_matrix: bool, weight_matrix: bool, as_dense: bool):
    generator = np.random.Generator(np.random.MT19937(seed))
    graph = mpg.graph.random_graph.gnp_random_mpg(n=n, p=p, seed=seed, method="fast", loops=True,
                                                  distribution="integers", low=0, high=10, endpoint=True)
    output = None
    if as_graph:
        output = graph
    else:
        if adj_matrix and weight_matrix:
            A = _as_tensor(nx.adjacency_matrix(graph, weight=None), as_dense=as_dense, shape_hint=(n, n))
            W = _as_tensor(nx.adjacency_matrix(graph, weight="weight"), as_dense=as_dense, shape_hint=(n, n))
            if as_dense:
                output = tf.stack([A, W], axis=0)
            else:
                output = tf.cast(_stack_sparse_tensors((2, n, n), A, W), dtype=tf.float32)

        elif adj_matrix:
            output = tf.cast(_as_tensor(nx.adjacency_matrix(graph, weight=None), as_dense=as_dense, shape_hint=(n, n)),
                             dtype=tf.float32)
        elif weight_matrix:
            output = tf.cast(
                _as_tensor(nx.adjacency_matrix(graph, weight="weight"), as_dense=as_dense, shape_hint=(n, n)),
                dtype=tf.float32)
    starting = tf.constant([generator.integers(0, n), generator.integers(0, 1, endpoint=True)])
    if target:
        # TODO: Add target
        return (output, starting, 1)
    else:
        return (output, 1)


def _generate_dense_instances(n, p, seed, cardinality: int, target: bool, weight_matrix: bool, flatten: bool):
    bernoulli: tfb.distributions.Distribution = tfb.distributions.Bernoulli(probs=p)
    # discrete=tfb.distributions.DiscreteUniform(low=0,high=10)
    shape = (n, n) if not flatten else (n * n,)
    W = tf.random.uniform(shape,-10, 11, dtype=tf.int32)
    adjacency_list=[]
    for k in range(n):
        adjacency_list.append(bernoulli.sample((n,)))
        while tf.math.reduce_all(adjacency_list[k]==0):
            adjacency_list[k] = bernoulli.sample((n,))
    A= tf.concat(adjacency_list,0)
    W = tf.multiply(A, W)
    vertex = tf.random.uniform((1,),0, n, dtype=np.int32)
    player = bernoulli.sample((1,))
    if flatten:
        if weight_matrix:
            output = tf.cast(tf.concat([A, W, vertex, player], axis=0), dtype=tf.float32)
        else:
            output = tf.cast(tf.concat([A, W, vertex, player], axis=0), dtype=tf.float32)
        if target:
            target_value=tf.py_function(lambda output:mpgwrapper.mpgcpp.winners_tensorflow_float_matrix_flattened_cxx(output.numpy().tolist()),inp=[output],Tout=tf.int32)
            target_value=tf.reshape(tf.ensure_shape(target_value,()),shape=(1,))
            return (output, tf.cast(target_value,dtype=tf.float32))
        return output
    else:
        if weight_matrix:
            output = tf.cast(tf.stack([A, W], axis=0), dtype=tf.float32)
        else:
            output = tf.cast(A, dtype=tf.float32)
        if target:
            return (output, tf.constant([vertex, player]), 1)
        return (output, tf.constant([vertex, player]))


class MPGGeneratedDenseDataset(tf.data.Dataset):

    def __new__(cls, n, p, cardinality=tf.data.INFINITE_CARDINALITY,
                target: bool = False, weight_matrix: bool = True, flatten=False,seed=None):
        if seed is None:
            seed = np.random.randint(0, 1<<32)
        shape = None
        if flatten:
            if weight_matrix:
                shape = (2 * n * n + 2,)
            else:
                shape = (n * n + 2,)
            signature = (tf.TensorSpec(shape=shape, dtype=tf.float32),)
        else:
            if weight_matrix:
                shape = (2, n, n)
            else:
                shape = (n, n)
            signature = (tf.TensorSpec(shape=shape, dtype=tf.float32), tf.TensorSpec(shape=(2,), dtype=tf.int32))
        if target:
            signature = (*signature, tf.TensorSpec(shape=()))

        generated: tf.data.Dataset
        if cardinality == tf.data.INFINITE_CARDINALITY:
            generated = tf.data.Dataset.counter(start=seed, step=1)
        else:
            generated = tf.data.Dataset.range(seed,seed+cardinality)
        return generated.map(
            lambda seed: _generate_dense_instances(n, p, seed, cardinality, target, weight_matrix, flatten),
            num_parallel_calls=tf.data.experimental.AUTOTUNE
        )
        #    range,
        #    args=(n,p,cardinality, target, weight_matrix,flatten),
        #    output_signature=signature
        # )

    def __init__(self, n, p):
        pass


class MPGGeneratedDataset(tf.data.Dataset):
    def _generator(n, p, cardinality: int, target: bool, as_graph: bool,
                   adj_matrix: bool, weight_matrix: bool, as_dense: bool):
        if cardinality == tf.data.INFINITE_CARDINALITY:
            seed = 0
            while True:
                yield _generate_instances(n, p, seed, cardinality, target, as_graph, adj_matrix, weight_matrix,
                                          as_dense)
                seed += 1
        else:
            for sample_idx in range(cardinality):
                yield _generate_instances(n, p, sample_idx, cardinality, target, as_graph, adj_matrix, weight_matrix,
                                          as_dense)

    def __new__(cls, n, p, cardinality=tf.data.INFINITE_CARDINALITY, target: bool = False, as_graph: bool = False,
                adj_matrix: bool = True, weight_matrix: bool = True, as_dense: bool = True):
        shape = None
        if as_graph:
            signature = tf.TensorSpec(shape=(), dtype=mpg.MeanPayoffGraph)
        else:
            if adj_matrix and weight_matrix:
                shape = (2, n, n)
            elif adj_matrix or weight_matrix:
                shape = (n, n)
            else:
                raise ValueError("Must specify at least one of adj_matrix or weight_matrix")
        if as_dense:
            TensorSpec = tf.TensorSpec
        else:
            TensorSpec = tf.SparseTensorSpec
        signature = (TensorSpec(shape=shape), tf.TensorSpec(shape=(2,), dtype=tf.int32))
        if target:
            signature = (*signature, tf.TensorSpec(shape=()))
        return tf.data.Dataset.from_generator(
            cls._generator,
            args=(n, p, cardinality, target, as_graph, adj_matrix, weight_matrix, as_dense),
            output_signature=signature
        )

    def __init__(self, n, p):
        pass


In [40]:
dataset=MPGGeneratedDenseDataset(10,0.5,weight_matrix=True,target=True,flatten=True,seed=0)
transformed=dataset.prefetch(1024).filter(lambda x,y:(y==0)[0] | (y==1)[0]).batch(64,drop_remainder=True,num_parallel_calls=12)
validation_dataset=MPGGeneratedDenseDataset(10,0.5,cardinality=64,weight_matrix=True,flatten=True,target=True,seed=653300).prefetch(1024).filter(lambda x,y:(y==0)[0] | (y==1)[0]).batch(32).cache()


In [41]:

import tensorflow.keras as keras
model=keras.Sequential([
    keras.layers.Flatten(),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(100,activation="relu"),
    keras.layers.Dense(50,activation="relu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(50,activation="relu"),
    keras.layers.Dense(15,activation="relu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(15,activation="relu"),
    keras.layers.Dense(1,"sigmoid")
])

In [ ]:
model.compile(
    optimizer="Adam",
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=["accuracy"],
)
model.fit(x=transformed,
    epochs=25,
    verbose='auto',
    steps_per_epoch=64,
    shuffle=True
         )

Epoch 1/25
64/64 [==============================] - 28s 365ms/step - loss: 0.8287 - accuracy: 0.4648
Epoch 2/25
64/64 [==============================] - 27s 429ms/step - loss: 0.6498 - accuracy: 0.6255
Epoch 3/25
64/64 [==============================] - 28s 434ms/step - loss: 0.6252 - accuracy: 0.6680
Epoch 4/25
64/64 [==============================] - 27s 429ms/step - loss: 0.6135 - accuracy: 0.6653
Epoch 5/25
 2/64 [..............................] - ETA: 26s - loss: 0.6139 - accuracy: 0.6719

In [6]:
for x,y in dataset.take(100):
    print(y)

tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(1, shape=(), dtype

TypeError: Tensor.__init__() missing 2 required positional arguments: 'value_index' and 'dtype'

In [111]:
graph = mpg.graph.random_graph.gnp_random_mpg(10, 0.5, seed=1, method="fast", loops=True,
                                              distribution="integers", low=0, high=10, endpoint=True)

In [123]:
A = tf.convert_to_tensor(nx.adjacency_matrix(graph, weight=None).todense())

In [181]:

coo = nx.adjacency_matrix(graph).tocoo()
indices = np.mat([coo.row, coo.col]).transpose()
Z=tf.SparseTensor(indices, coo.data, (10,10))

TypeError: binomial() takes at least 2 positional arguments (0 given)

In [28]:
import numpy as np
import tensorflow as tf
import mpg.graph.random_graph
import networkx as nx


def _convert_sparse_matrix_to_sparse_tensor(X, shape_hint):
    coo = X.tocoo()
    indices = np.mat([coo.row, coo.col]).transpose()
    return tf.SparseTensor(indices, coo.data, shape_hint)


def _stack_sparse_tensors(shape_hint, *A):
    indices = []
    values = []
    for i, Z in enumerate(A):
        for L, v in zip(Z.indices, Z.values):
            indices.append([i, *L])
            values.append(v)
    return tf.sparse.SparseTensor(indices, values, shape_hint)


def _as_tensor(A, as_dense: bool, shape_hint=None):
    if as_dense:
        return tf.convert_to_tensor(A.todense())
    else:
        return _convert_sparse_matrix_to_sparse_tensor(A, shape_hint)


def _generate_instances(seed, cardinality: int, target: bool, as_graph: bool,
                        adj_matrix: bool, weight_matrix: bool, as_dense: bool, dtype=np.float32):
    graph = mpg.graph.random_graph.gnp_random_mpg(10, 0.5, seed=seed, method="fast", loops=True,
                                                  distribution="integers", low=0, high=10, endpoint=True)
    n = len(graph)
    output = None
    generator= np.random.Generator(np.random.MT19937(seed))
    if as_graph:
        output = graph
    else:
        if adj_matrix and weight_matrix:
            A = _as_tensor(nx.adjacency_matrix(graph, weight=None), as_dense=as_dense, shape_hint=(n, n))
            W = _as_tensor(nx.adjacency_matrix(graph, weight="weight"), as_dense=as_dense, shape_hint=(n, n))
            if as_dense:
                output = tf.stack([A, W], axis=0)
            else:
                output = tf.cast(_stack_sparse_tensors((2, n, n), A, W), dtype=dtype)


        elif adj_matrix:
            output = _as_tensor(nx.adjacency_matrix(graph, weight=None), as_dense=as_dense, shape_hint=(n, n))
        elif weight_matrix:
            output = _as_tensor(nx.adjacency_matrix(graph, weight="weight"), as_dense=as_dense, shape_hint=(n, n))
    starting=np.array(generator.integers(0, n, endpoint=False), generator.integers(0, 1, endpoint=True))
    if target:
        # TODO: Add target
        return output, starting
    else:
        return output, starting


class MPGGeneratedDataset(tf.data.Dataset):
    def _generator(cardinality: int, target: bool, as_graph: bool,
                   adj_matrix: bool, weight_matrix: bool, as_dense, dtype):
        if cardinality == tf.data.INFINITE_CARDINALITY:
            seed = 0
            while True:
                yield _generate_instances(seed, cardinality, target, as_graph, adj_matrix, weight_matrix, as_dense)
                seed += 1
        else:
            for sample_idx in range(cardinality):
                yield _generate_instances(sample_idx, cardinality, target, as_graph, adj_matrix, weight_matrix,
                                          as_dense)

    def __new__(cls, cardinality=tf.data.INFINITE_CARDINALITY, target: bool = False, as_graph: bool = False,
                adj_matrix: bool = True, weight_matrix: bool = True, as_dense: bool = True, dtype=np.float32):
        shape = None
        if as_graph:
            signature = tf.TensorSpec(shape=(), dtype=mpg.MeanPayoffGraph)
        else:
            if adj_matrix and weight_matrix:
                shape = (2, None, None)
            elif adj_matrix or weight_matrix:
                shape = (None, None)
            else:
                raise ValueError("Must specify at least one of adj_matrix or weight_matrix")
        if as_dense:
            TensorSpec = tf.TensorSpec
        else:
            TensorSpec = tf.SparseTensorSpec
        signature = (TensorSpec(shape=shape, dtype=tf.float32),tf.TensorSpec(shape=(2,),dtype=tf.int16))
        if target:
            signature = (*signature, tf.TensorSpec(shape=(), dtype=dtype))
        return tf.data.Dataset.from_generator(
            cls._generator,
            args=(cardinality, target, as_graph, adj_matrix, weight_matrix, as_dense, dtype),
            output_signature=signature
        )

2023-04-02 15:36:19.793565: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [198]:
zip(range(2),Z.indices)

tf.Tensor([0 0], shape=(2,), dtype=int64)
tf.Tensor([0 5], shape=(2,), dtype=int64)
tf.Tensor([0 8], shape=(2,), dtype=int64)
tf.Tensor([0 9], shape=(2,), dtype=int64)
tf.Tensor([1 0], shape=(2,), dtype=int64)
tf.Tensor([1 2], shape=(2,), dtype=int64)
tf.Tensor([1 3], shape=(2,), dtype=int64)
tf.Tensor([1 5], shape=(2,), dtype=int64)
tf.Tensor([1 7], shape=(2,), dtype=int64)
tf.Tensor([1 8], shape=(2,), dtype=int64)
tf.Tensor([1 9], shape=(2,), dtype=int64)
tf.Tensor([2 0], shape=(2,), dtype=int64)
tf.Tensor([2 2], shape=(2,), dtype=int64)
tf.Tensor([2 3], shape=(2,), dtype=int64)
tf.Tensor([2 4], shape=(2,), dtype=int64)
tf.Tensor([2 7], shape=(2,), dtype=int64)
tf.Tensor([2 8], shape=(2,), dtype=int64)
tf.Tensor([3 0], shape=(2,), dtype=int64)
tf.Tensor([3 2], shape=(2,), dtype=int64)
tf.Tensor([3 3], shape=(2,), dtype=int64)
tf.Tensor([3 5], shape=(2,), dtype=int64)
tf.Tensor([3 7], shape=(2,), dtype=int64)
tf.Tensor([3 8], shape=(2,), dtype=int64)
tf.Tensor([4 1], shape=(2,), dtype

In [238]:
tf.cast(Z,tf.float64)

SparseTensor(indices=tf.Tensor(
[[0 0]
 [0 5]
 [0 8]
 [0 9]
 [1 0]
 [1 2]
 [1 3]
 [1 5]
 [1 7]
 [1 8]
 [1 9]
 [2 0]
 [2 2]
 [2 3]
 [2 4]
 [2 7]
 [2 8]
 [3 0]
 [3 2]
 [3 3]
 [3 5]
 [3 7]
 [3 8]
 [4 1]
 [4 4]
 [4 5]
 [4 7]
 [5 0]
 [5 2]
 [5 3]
 [5 5]
 [5 6]
 [6 1]
 [6 2]
 [6 4]
 [6 5]
 [6 7]
 [6 8]
 [6 9]
 [7 2]
 [7 4]
 [7 6]
 [7 7]
 [7 8]
 [7 9]
 [8 1]
 [8 2]
 [8 3]
 [8 7]
 [8 8]
 [8 9]
 [9 1]
 [9 4]], shape=(53, 2), dtype=int64), values=tf.Tensor(
[ 2.  8.  1.  9.  9.  1.  5.  6.  3.  1. 10.  5.  4.  1.  1.  7.  1.  0.
  3.  4. 10. 10.  8.  0.  0.  3.  3.  6. 10.  8. 10.  1.  9.  2. 10.  5.
  0.  9.  0.  3. 10.  4.  8.  3.  8.  2.  4.  0.  9.  6.  7.  6.  1.], shape=(53,), dtype=float64), dense_shape=tf.Tensor([10 10], shape=(2,), dtype=int64))

In [2]:
from numba import jit
import networkx as nx
import numpy as np
from typing import Union, TypeVar, Type
import mpg.games.mpg as mpg


1.19 s ± 32.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [23]:
import mpg.graph.random_graph as rg
G= rg.gnp_random_mpg(500,0.5,seed=27,distribution="integers",low=-10,high=10,endpoint=True)

In [27]:
%timeit mpg.optimal_strategy_pair(G)

20.2 s ± 170 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [25]:
import mpg.wrapper as wrapper
%timeit wrapper.optimal_strategy_pair(G)

664 ms ± 63.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [110]:

def _get_winner(A,W,vertex,player):
    edges = []
    for i in range(A.shape[0]):
        for j in range(A.shape[1]):
            if A[i,j] == 1:
                edges.append((i,j,W[i,j]))
    C=mpgwrapper.mpgcpp.winners_double_edges_cxx(edges)
    return C[player][vertex]

In [ ]:
A=tf